# Geometric optics[(solutions)](../../ph3080_data/case_study_solutions/08.Geometric_optics-answers.ipynb)
<pre>
PH3080 (Computational Physics)
Michael Mazilu and Aly Gillies, University of St Andrews
ph3080@st-andrews.ac.uk 
Version: 2021; (CC BY-NC 2.0) 
</pre>

In [1]:
%matplotlib widget
import numpy as np

from scipy.optimize import minimize
from scipy.linalg import eig

import matplotlib.pyplot as plt

## Physics background

<img src="../img/osys.png" width =750/>

In this study we will model the propagation of rays through simple optical systems consisting of lenses and curved mirrors.
To do this, we use the small angle approximation, i.e. the rays propagate in a *paraxial regime* (close to the optic-axis), where the following relationship is valid:

$$\begin{pmatrix}x_2\\ x'_2\end{pmatrix}=\begin{pmatrix}A&B\\C&D\end{pmatrix} \begin{pmatrix}x_1\\ x'_1\end{pmatrix} \label{abcd}$$

In this, $x_1$ and $x'_1$ are the height and gradient of the incident ray with respect to the optic-axis at position $z_1$. Similarly, $x_2$ and $x'_2$ are the height and gradient of the transmitted ray at position $z_2$.  The matrix elements $A$, $B$, $C$ and $D$  describe an optical system consisting of one or more optical components. 
In optics, these matrices are called **ABCD** matrices. 
Of particular interest to us are the following fundamental optical components:

| Free-space distance $d$ | Thin lens | Curved mirror | Object/Imaging plane|
| --- | --- | --- | --- |
| $$\begin{pmatrix}1&d\\0&1\end{pmatrix}$$ | $$\begin{pmatrix}1&0\\\displaystyle -\frac{1}{f}&1\end{pmatrix}$$ | $$\begin{pmatrix}1&0\\\displaystyle -\frac{2}{R_m}&1\end{pmatrix}$$ | $$\begin{pmatrix}1&0\\0&1\end{pmatrix}$$ |

where $f$ is the focal length of the thin lens, with $f>0$ for a convex lens,
and $R_m$ is the radius of curvature of the mirror with $R_m>0$ for a concave mirror.

In our discussions we will call the vector  ($x$, $x'$) the *ray-state* vector.

Propagation through multiple optical components allows for the modelling of microscopes, telescopes and laser cavities, and can be achieved via matrix multiplication.

## Programming techniques
### Linear algebra
Initially, we define a function that takes distance, $d$, as an argument and generates the ABCD matrix corresponding to a free-space propagation distance. In this case study, we will assume all distanced are in the same units, in this case  millimetres, unless otherwise stated.

In [2]:
def dist(d):
    ABCD = np.identity(2) # start with the identity matrix then modify
    ABCD[0,1]= d          # change value in first row, second column
    return ABCD
print(dist(5))

[[1. 5.]
 [0. 1.]]


### Define additional ABCD matrices
Define functions for the other two fundamental optical components, naming them `lens` and `mirror`. Define a function called `plane` that returns the identity matrix for use as a placeholder for the object and imaging planes.

### Verification through a simple system
Consider a ray, defined by $z_1=0$, $x_1=40$ and $x'_1=-0.2$, propagating through a lens positioned at $z=300$ with focal length $f=180$. 

Using the matrix multiplication method, verify that at position $z=450$ the ray's height and gradient are $-33.3333$ and $-0.0888889$ respectively. Remember that the ray has to go through free space up to the lens ($0$ to $300$), then through the lens, then through free space to the requested position ($300$ to $450$).

### Draw the optics
Use `matplotlib.pyplot` to create a plot showing the lens, the optic-axis, and the ray before and after the lens, up to the point of observation. Use `plt.plot` and a dashed line for the lens (diameter 50), a solid line for the ray segments and an `axhline` for the optic-axis.

### Add another lens
Add a second lens, positioned at $z=400$ with focal length $f=100$, and recalculate the final height and gradient of our original ray at position $z=450$.

### Compare with combined matrix
Verify that you get the same final height and gradient when using a combined ABCD matrix for the whole two-lens system. Make sure that you multiply the ABCD matrices in the correct order and take into account the free-space propagation between elements. 

### Predefined functions
To continue with the case study we will use a few purpose-built functions. These are defined in the cell below.

As inputs, these functions use single rays and single optical elements defined as dictionaries. Multiple rays and multiple optical elements are defined as lists of such dictionaries.

In [8]:
def new_ray(x, dx, z=0, color=(1,0,0)): # define a new ray
    return {
        "state": np.array([x, dx]), # height and gradient of ray
        "z1":z, "z2":np.inf,        # start position and end position of ray (default end is infinity)
        "color":color               # colour that will be used to plot ray
    }

def new_opt(trans, z, diam=0): # define a new optical element with ABCD matrix, position and aperture size
    return {"trans":trans, "z":z, "diam":diam} 

def prop_one(rays, opt):
    """
    Propagate a list of rays through a single optical element, 
    checking first that they actually intercept the optical element.
    
    rays (list): a list of rays
    opt (dictionary): an optical element
    
    returns (list): a list of rays
    """
    # extract the details of the optical element
    z = opt["z"]
    trans = opt["trans"]
    diam = opt["diam"]
    
    #define an empty list in which to store the propagated rays
    out_rays = []
    
    # propagate the rays
    for ray in rays:  # take each ray at a time from the list of rays
        # make a copy of the ray to prevent changing the input
        in_ray = ray.copy()
        # extract the ray's properties
        state = in_ray["state"]
        z1 = in_ray["z1"]
        z2 = in_ray["z2"]
        color = in_ray["color"]
        # check the ray is still active and begins before the optical element
        if z2 == np.inf and z1 <= z:
            # if the ray reaches the optical element then stop it there
            in_ray["z2"] = z
            
            # propagate the ray *to* the optical element
            state1 = dist(z-z1)@state
            # check if the ray gets through the aperture, if it does 
            # then calculate a new ray emitted from the optical emement
            if np.abs(state1[0]) < diam/2 :
                # calculate the effect of the optical element
                state2 = trans@state1
                # and create a new ray
                out_ray = new_ray(*state2, z, color)
                # and put it in our list of rays 
                out_rays.append(out_ray)
        
        # always put the incident ray in our list (with modified z2 if appropriate)
        out_rays.append(in_ray)
    return out_rays

def prop_rays(rays, optics):
    """
    propagate a list of rays through a list of optical elements
    
    rays (list): a list of rays
    opts (list): an optical element
    """
    for opt in optics: # take each optical element in turn
        rays = prop_one(rays, opt) # propogate all rays through chosen optic
    return rays

def combined_ABCD(optics):
    """
    Calculate the combined transformation ABCD matrix of a list of optical elements
    
    optics (list): a list of optical elements
    
    returns (np.array): the net transformation
    """
    
    trans = np.identity(2) # start with identity matrix
    z = optics[0]["z"]     # prepare distance to next (first) optical element
    
    for opt in optics:     # for each element in the optical configuration
        freesp_ABCD = dist(opt["z"]-z)
        trans = opt["trans"]@freesp_ABCD@trans # include transit to and through optic
        z = opt["z"]       # prepare distance to next optical element 
    return trans


def show_optics(rays, optics):
    """
    Visualise an optical setup and the rays propagated through it.
    
    rays (list): a list of rays
    optics (list): a list of optical elements
    
    return: None
    """
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    ax.set_title("Optical system")
    ax.set_xlabel("z (mm)")
    ax.set_ylabel("x (mm)")
    
    ax.axhline(0, c="lightgray", zorder=0, linestyle="dashed") # draw optic-axis
    
    for opt in optics:  # for each optic in turn
        z = opt["z"]
        diam = opt["diam"]
        if diam != 0:    
            ax.axvline(z, c="k", zorder=0)                                        # draw an infinite, vertical, black line at position of optic
            ax.plot([z,z], [diam/2, -diam/2], c="w", linestyle="-", zorder=0)     # draw a white line for the aperture
            ax.plot([z,z], [diam/2, -diam/2], c="k", linestyle="--", zorder=10)   # make the aperture dotted for visibility
                    
    z_max = ax.get_xlim()[1] # get maximum x from plot of optical elements
    z_max = z_max + z_max/10 # extend 10% beyond max x from plot to distance past last optic
    for ray in rays:
        state = ray["state"]
        z1 = ray["z1"]
        z2 = ray["z2"]
        color = ray["color"]
        
        if z2 == np.inf:
            z2 = z_max
            linestyle = "--"
        else:
            linestyle = "-"
            
        x1 = state[0]
        x2 = state[0] + state[1]*(z2-z1) # use gradient and separation to change height
             
        ax.plot([z1,z2], [x1, x2], c=color, zorder=5, linestyle=linestyle)

In [9]:
# a ray as a dictionary
print(new_ray(2, 0.01)) # ray with height 2mm, gradient 0.01

# an optical system as a list of dictionaries - a telescope
obj = new_opt(plane(), 0, 100)
lens1 = new_opt(lens(200), 200, 25)
lens2 = new_opt(lens(400), 800, 25)
end = new_opt(plane(), 1200)

tele = [obj, lens1, lens2, end]
print(tele)

{'state': array([2.  , 0.01]), 'z1': 0, 'z2': inf, 'color': (1, 0, 0)}
[{'trans': array([[1., 0.],
       [0., 1.]]), 'z': 0, 'diam': 100}, {'trans': array([[ 1.   ,  0.   ],
       [-0.005,  1.   ]]), 'z': 200, 'diam': 25}, {'trans': array([[ 1.    ,  0.    ],
       [-0.0025,  1.    ]]), 'z': 800, 'diam': 25}, {'trans': array([[1., 0.],
       [0., 1.]]), 'z': 1200, 'diam': 0}]


### Create a bundle of parallel rays
Create a list of parallel rays named `rays` with $x$ ranging from $-5$ to $5$ in steps of $1$, all with gradient $0.01$. This list corresponds to a bundle of upward-propagating parallel rays.

### Propagate rays through telescope
Use `prop_rays(rays,tele)` to calculate the propagation of the bundle of rays defined by the list `rays` through the telescope `tele`. Name the list of rays output from the function `p_rays`.

### Draw propagation through telescope
Use the `show_optics` function to display the propagated rays, `p_rays`, as they pass through the telescope.

<a id="verifyABCD"></a>
### Verify combined ABCD matrix
Using matrix multiplication (@ in Python), calculate the ABCD matrix for the whole telescope. Check your output against that calculated using the `combined_ABCD` function.

### Imaging systems
The following code defines a point source with rays emerging at varying gradients.

In [14]:
rays = []
for dx in np.arange(-0.01, 0.01, 0.001):
    rays.append(new_ray(2,dx))

### Plot imaging in telescope
Display the propagation of the `rays` from the point source through the telescope defined by `tele`. Notice that all the rays are focused by the telescope to a single point. This telescope configuration is called a 4F arrangement, i.e. distance $f_1$ between the object and the first lens, distance $f_1 + f_2$ between the two lenses, and distance $f_2$ between the second lens and the imaging plane.

## Study
We want to study the properties of an optical system as a function of its constituent parts. We will look at imaging in an eye, discuss magnification factor, and study rays in periodic focusing systems.

### Model imaging of an eye
Model an eye as a lens, positioned at $z=400$ with a focal length of $17$, and a retina (imaging plane) at a distance of 17 behind the lens (positioned at $z=417$). Use $10$ as the iris diameter of the eye. Use the `show_optics` function to model the eye looking at a point-source object at $z=0$ and $x=10$ (set the aperture diameter as 100) with a ray bundle of gradient varying from $-0.05$ to $0.00$ in steps of $0.002$. Is the object in focus? You may wish to zoom in to examine the focal point more closely; use the "Zoom to rectangle" option (little square control) on the plot to zoom (assuming you are using `%matplotlib widget`).

### Quantifying focus
Define a function called `focus_variance` that takes the focal length of the eye's lens, and the object-to-eye distance as arguments. The function should return the variance (`np.var`) of the distribution of the rays striking the retina. Plot the focus variance as a function of eye focal-length for three different object distances or $400$, $1000$ and $2000$.

### Find best focus
Find the minimum variance for each of the three object positions ($400$, $1000$ and $2000$) and determine the eye focal length producing these best focii. Use the `show_optics` function to show the propagation of a point-source object at $z=0$ and $x=10$ (set aperture diameter as 100) with a ray bundle of gradient varying from $-0.05$ to $0.00$ in steps of $0.001$.

In [18]:
focal_length_400 =  minimize(focus_variance, 17, args=(400,), tol=1E-10).x[0]
focal_length_1000 = minimize(focus_variance, 17, args=(1000,), tol=1E-10).x[0]
focal_length_2000 = minimize(focus_variance, 17, args=(2000,0.0001), tol=1E-10).x[0]

print(f"  400 mm focal length: {focal_length_400:0.4f}\n",\
      f"1000 mm focal length: {focal_length_1000:0.4f}\n", \
      f"2000 mm focal length: {focal_length_2000:0.4f}") #printed with 4 decimal places

  400 mm focal length: 16.3070
 1000 mm focal length: 16.7158
 2000 mm focal length: 16.8567


### Combined ABCD matrix of imaging systems
Calculate the overall ABCD matrices for each of the object positions ($400$, $1000$ and $2000$) when in best focus (minimum variance). You should have three ABCD matrices with $B \approx 0$. Indeed, any optical system where $B=0$ will be an imaging system.

### Magnification factor
Taking any one of these ABCD matrices, determine the relationship between the height of the object and the image size on the retina. What is the magnification factor in relationship to the ABCD matrix?

### 4F system magnification
Calculate the magnification for a $4F$ optical imaging telescope defined by two lenses with focal length $f_1$ and $f_2$ and check against that [calculated earlier](#verifyABCD).

Verify that the magnitude of this magnification factor is equal to $f_2/f_1$. Can you explain what the sign being negative means?

### Periodic optical system
Consider a periodically repeating optical system where each period consists of a thin lens of focal length $f$ and a distance $d$ of free-space propagation. Display the height of a ray starting with $(x_1 , x_1' ) = (1, 0)$ as it propagates through the first $100$ periods as a function of the period number. Do this for $f = 10$; $d = 1$ and for $f = 1$; $d = 10$. Comment on what you observe, and why this is the case.

### ABCD eigenvalue
Determine the eigenvalue of the single-period ABCD matrix and compare the absolute value of these in the two cases. What is the physical meaning of the eigenvalue with respect to the $x$-distance of the ray propagating through the system?

### Stability region plot
Use the matplotlib `contourf` function to display the region of stability for our periodically repeating optical system with $d$ varying between $0$ and $10$ and $f$ between $-10$ and $10$. This is called a “lens&nbsp;equivalent&nbsp;waveguide”.

Expect this to take approx $10-15s$ to compute.

### Laser cavity stability
Consider an optical cavity consisting of two spherical curved mirrors with radii $R_1$ and $R_2$ separated by the distance $L$. A round trip in the cavity corresponds to one period. Considering one full period of the cavity, plot the region of stability for $L = 1$ in the parameter space $R_1$ and $R_2$ varying each between $-4$ and $4$.

## Advanced problem

The stability region is usually represented as a function of parameters $g_1$ and $g_2$, where $g_1 = 1 − L/R_1$ and $g_2 = 1 − L/R_2$. Considering the case where $L = 1$, write $R_1$ and $R_2$ as a function of $g_1$ and $g_2$, and re-plot the region of stability with $g_1$ and $g_2$ varying between $-4$ and $4$. Check your graph against https://en.wikipedia.org/wiki/Optical_cavity#Stability.

## Function glossary

- scipy.minimize
- plt.contourf
- np.meshgrid
- dictionaries

